In [1]:
import pandas as pd
import numpy as np
# from pymongo import MongoClient

# Initial cleanup

In [2]:
df = pd.read_csv('../data/nfl_plays.csv')

/Users/Philip/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (15,16,17,22,27,33,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.columns

Index([u'GameId', u'GameDate', u'Quarter', u'Minute', u'Second',
       u'OffenseTeam', u'DefenseTeam', u'Down', u'ToGo', u'YardLine',
       u'Unnamed: 10', u'SeriesFirstDown', u'Unnamed: 12', u'NextScore',
       u'Description', u'TeamWin', u'Unnamed: 16', u'Unnamed: 17',
       u'SeasonYear', u'Yards', u'Formation', u'PlayType', u'IsRush',
       u'IsPass', u'IsIncomplete', u'IsTouchdown', u'PassType', u'IsSack',
       u'IsChallenge', u'IsChallengeReversed', u'Challenger', u'IsMeasurement',
       u'IsInterception', u'IsFumble', u'IsPenalty', u'IsTwoPointConversion',
       u'IsTwoPointConversionSuccessful', u'RushDirection', u'YardLineFixed',
       u'YardLineDirection', u'IsPenaltyAccepted', u'PenaltyTeam', u'IsNoPlay',
       u'PenaltyType', u'PenaltyYards', u'Unnamed: 45'],
      dtype='object')

In [4]:
# Remove unnamed columns. Most are empty seperators
df = df.drop([u'Unnamed: 10', u'Unnamed: 12', u'Unnamed: 16', u'Unnamed: 17', 
              u'Unnamed: 45','Challenger'], axis=1)

In [5]:
# Turn gamedate into datetimes
df['GameDate'] = df['GameDate'].apply(pd.to_datetime)

In [6]:
# Replace null values of 'SeasonYear' with 2013
mask = df['SeasonYear'].isnull()
df.loc[mask, 'SeasonYear'] = 2013.0

mask = df['SeasonYear'] == 0
df.loc[mask, 'SeasonYear'] = 2015.0

In [7]:
# Replace null values of 'Yards' with 0.0
mask = df['Yards'].isnull()
df.loc[mask, 'Yards'] = 0.0

In [8]:
# Replace null values of 'PlayType' with 'RUSH', they are wildcat plays
mask = df['PlayType'].isnull()
df.loc[mask, 'PlayType'] = 'RUSH'

mask = df['PlayType'] == '0'
df.loc[mask, 'PlayType'] = 'PASS'

mask = df['PlayType'] == 'UNDER CENTER'
df.loc[mask, 'PlayType'] = 'PASS'

mask = df['PlayType'] == 'SHOTGUN'
df.loc[mask, 'PlayType'] = 'PASS'
df.loc[mask, 'IsPass'] = 1

In [9]:
# Replace playType two point conversion with rush or pass
mask = (df['PlayType'] == 'TWO-POINT CONVERSION') & (df['Description'].str.contains('PASS'))
df.loc[mask, 'PlayType'] = 'PASS'
df.loc[mask, 'IsPass'] = 1
df.loc[mask, 'IsTwoPointConversion'] = 1

mask = (df['PlayType'] == 'TWO-POINT CONVERSION') & (df['Description'].str.contains('RUSH'))
df.loc[mask, 'PlayType'] = 'RUSH'
df.loc[mask, 'IsRush'] = 1
df.loc[mask, 'IsTwoPointConversion'] = 1

In [10]:
# Replace null values of 'IsPass' with 0
mask = df['IsPass'].isnull()
df.loc[mask, 'IsPass'] = 0

In [11]:
# Clean up IsPass values that should be equal to 1
mask = (df['PlayType'] == 'PASS') & (df['IsPass'] != 1)
df.loc[mask, 'IsPass'] = 1

mask_2 = (df['PlayType'] != 'PASS') & (df['IsPass'] == 1)
df.loc[mask_2, 'IsPass'] = 0

In [12]:
# Clean up IsRush values
mask = df['IsRush'] == 'PASS'
df.loc[mask, 'IsRush'] = 0

mask = df['IsRush'] == '0'
df.loc[mask, 'IsRush'] = 0

mask = df['IsRush'] == '1'
df.loc[mask, 'IsRush'] = 1

In [13]:
# Replace null values of 'IsTouchdown' with 0
mask = df['IsTouchdown'].isnull()
df.loc[mask, 'IsTouchdown'] = 0

In [14]:
# Replace null values of 'PassType' with 0, also kicks were being missclassified as PASS
mask = df['PlayType'] != 'PASS'
df.loc[mask, 'PassType'] = 0

mask = (df['PlayType'] == 'PASS') & df['Description'].str.contains('KICKS')
df.loc[mask, 'PassType'] = 0
df.loc[mask, 'PlayType'] = 'PUNT'

mask = (df['PlayType'] == 'PASS') & df['Description'].str.contains('FIELD GOAL')
df.loc[mask, 'PassType'] = 0
df.loc[mask, 'PlayType'] = 'FIELD GOAL'

mask = (df['PlayType'] == 'PASS') & df['Description'].str.contains('EXTRA POINT')
df.loc[mask, 'PassType'] = 0
df.loc[mask, 'PlayType'] = 'EXTRA POINT'

mask = (df['PlayType'] == 'PASS') & df['Description'].str.contains('KNEELS')
df.loc[mask, 'PassType'] = 0
df.loc[mask, 'PlayType'] = 'QB KNEEL'

mask = (df['PlayType'] == 'PASS') & df['Description'].str.contains('END GAME')
df.loc[mask, 'PassType'] = 0
df.loc[mask, 'PlayType'] = 'NO PLAY'

In [15]:
# Replace null values of 'IsIncomplete' with 0
mask = df['IsIncomplete'].isnull()
df[mask] = df[mask].fillna(0)

In [16]:
# Replace all of the null values in one faulty row with 0's
mask = df['IsSack'].isnull()
df[mask] = df[mask].fillna(0)

In [17]:
# Replace null values of 'IsMeasurement' with 0
df['IsMeasurement'] = df['IsMeasurement'].fillna(0)

In [18]:
# Extract missing values of YardLineFixed from YardLine
mask = df['YardLineFixed'].isnull()
df.loc[mask, 'YardLineFixed'] = 100 - df[mask]['YardLine'] 

In [19]:
# Replace null values of 'IsNoPlay' with 0
df['IsNoPlay'] = df['IsNoPlay'].fillna(0)

In [20]:
# Replace null values of 'PenaltyYards' with 0
df['PenaltyYards'] = df['PenaltyYards'].fillna(0)

In [21]:
# Drop the rest and include description
df.dropna(subset=['OffenseTeam', 'Description'], how='any', inplace = True)

In [22]:
# Replace null values of 'Formation' with 0
df['Formation'] = df['Formation'].fillna(0)

In [23]:
# Correct mislabeled rush plays with the right RushDirection
mask = (df['RushDirection'].isnull()) & (df['PlayType'] == 'RUSH') & (df['Description'].str.contains('CENTER'))
df.loc[mask, 'RushDirection'] = 'CENTER'

mask = (df['RushDirection'].isnull()) & (df['PlayType'] == 'RUSH') & (df['Description'].str.contains('LEFT TACKLE'))
df.loc[mask, 'RushDirection'] = 'LEFT TACKLE'

mask = (df['RushDirection'].isnull()) & (df['PlayType'] == 'RUSH') & (df['Description'].str.contains('RIGHT GUARD'))
df.loc[mask, 'RushDirection'] = 'RIGHT GUARD'

mask = (df['RushDirection'].isnull()) & (df['PlayType'] == 'RUSH') & (df['Description'].str.contains('LEFT GUARD'))
df.loc[mask, 'RushDirection'] = 'LEFT GUARD'

mask = (df['RushDirection'].isnull()) & (df['PlayType'] == 'RUSH') & (df['Description'].str.contains('RIGHT TACKLE'))
df.loc[mask, 'RushDirection'] = 'RIGHT TACKLE'

mask = (df['RushDirection'].isnull()) & (df['PlayType'] == 'RUSH') & (df['Description'].str.contains('RIGHT END'))
df.loc[mask, 'RushDirection'] = 'RIGHT END'

mask = (df['RushDirection'].isnull()) & (df['PlayType'] == 'RUSH') & (df['Description'].str.contains('LEFT END'))
df.loc[mask, 'RushDirection'] = 'LEFT END'

# fill the rest with 0
mask = df['RushDirection'].isnull()
df[mask] = df[mask].fillna(0)

In [24]:
# Replace null values of 'PenaltyTeam' with 0
df['PenaltyTeam'] = df['PenaltyTeam'].fillna(0)

In [25]:
# Replace null values of 'PenaltyType' with 0
df['PenaltyType'] = df['PenaltyType'].fillna(0)

In [26]:
# Turn descriptions to unicode to be able to upload to mongodb
# df['Description'] = df['Description'].apply(lambda x: unicode(x, 'utf-8', errors="ignore"))

# To upload data to mongodb, there are limits, will have to divy by team or season and upload to different tables
# db_cilent = MongoClient()
# db = db_cilent['NFL']
# table = db['plays']
# odo(df, db.table)

In [27]:
# Replace offenseteam/defenseteam where 'SD' with 'LAC'
mask = df['OffenseTeam'] == 'SD'
df.loc[mask, 'OffenseTeam'] = 'LAC'

mask = df['DefenseTeam'] == 'SD'
df.loc[mask, 'DefenseTeam'] = 'LAC'

In [28]:
# Create Team1_Team2 column to join weather data on
def join_teams(values):
    return "_".join(sorted(values))

df['Team1_Team2'] = df[[u'OffenseTeam', u'DefenseTeam']].apply(join_teams, axis=1)

# Import scraped weather data, clean, join

In [29]:
weather_df = pd.read_csv('../data/weather.csv')

In [30]:
# Turn GameDate to datetime
weather_df['GameDate'] = weather_df['GameDate'].apply(pd.to_datetime)

In [31]:
# Change type from unicode to string
weather_df['Team1_Team2'] = weather_df['Team1_Team2'].astype(str)

In [32]:
# Drop stadium, corrupt data
weather_df = weather_df.drop('Stadium', axis=1)

In [33]:
# Remove '%' from humidity column
weather_df['Humidity'] = weather_df['Humidity'].str.replace('%', '')

In [34]:
# Replace visibility null values with visibility mean
mask = weather_df['Visibility'].isnull()
weather_df.loc[mask, 'Visibility'] = weather_df['Visibility'].mean()

In [35]:
# Replace wind null values with wind mean
mask = weather_df['Wind'].isnull()
weather_df.loc[mask, 'Wind'] = weather_df['Wind'].mean()

In [36]:
# Replace weather category null values with clear (mode)
mask = weather_df['Weather_cat'].isnull()
weather_df.loc[mask, 'Weather_cat'] = 'Clear'

In [37]:
# Change grass subcategories to 'grass' value for surface feature
mask = (weather_df['Surface'].isnull()) | (weather_df['Surface'] == 'Bermuda') | \
        (weather_df['Surface'] == 'Bluegrass') | (weather_df['Surface'] == 'Kentucky') | \
            (weather_df['Surface'] == 'Natural')
    
weather_df.loc[mask, 'Surface'] = 'Grass'

In [38]:
# Change turf subcategories to 'Fieldturf' value for surface feature
mask = (weather_df['Surface'] == 'A-Turf') | \
        (weather_df['Surface'] == 'UBU') | (weather_df['Surface'] == 'FieldTurf') | \
            (weather_df['Surface'] == 'RealGrass')
    
weather_df.loc[mask, 'Surface'] = 'Fieldturf'

In [39]:
# Create major weather categories
mask = weather_df['Weather_cat'].str.contains('Cloud')
weather_df.loc[mask, 'Weather_cat'] = 'Cloudy'

mask = weather_df['Weather_cat'].str.contains('Rain') | weather_df['Weather_cat'].str.contains('Drizzle') | \
        weather_df['Weather_cat'].str.contains('Showers') | weather_df['Weather_cat'].str.contains('storm')
weather_df.loc[mask, 'Weather_cat'] = 'Rain'

mask = weather_df['Weather_cat'].str.contains('Fair') | weather_df['Weather_cat'].str.contains('Clear') | \
        weather_df['Weather_cat'].str.contains('Sunny') | weather_df['Weather_cat'].str.contains('Dry') | \
            weather_df['Weather_cat'].str.contains('Breezy') | weather_df['Weather_cat'].str.contains('Humid')
weather_df.loc[mask, 'Weather_cat'] = 'Clear'

mask = weather_df['Weather_cat'].str.contains('Fog') | weather_df['Weather_cat'].str.contains('Overcast')
weather_df.loc[mask, 'Weather_cat'] = 'Overcast' 

mask = weather_df['Weather_cat'].str.contains('Snow') | weather_df['Weather_cat'].str.contains('Wintry Mix') | \
        weather_df['Weather_cat'].str.contains('Flurries')
weather_df.loc[mask, 'Weather_cat'] = 'Snow'

In [40]:
# Merge original df with weather df
combined_df = pd.merge(df, weather_df, how='left', left_on = ['GameDate', 'Team1_Team2'], right_on = ['GameDate', 'Team1_Team2'])

# Feature Engineering, add scraped coach data

In [41]:
# Create columns to keep track of scoring plays. Will be used to calculate scores
mask = (combined_df['Description'].str.contains('TOUCHDOWN')) | (combined_df['IsTouchdown'] == 1)
combined_df['IsTouchdown'] = np.where(mask, 1, 0)

mask_2 = combined_df['Description'].str.contains('FIELD GOAL IS GOOD')
combined_df['IsFieldGoal'] = np.where(mask_2, 1, 0)

mask_3 = combined_df['Description'].str.contains('EXTRA POINT IS GOOD')
combined_df['IsExtraPoint'] = np.where(mask_3, 1, 0)

mask_4 = ((combined_df['Description'].str.contains('TWO-POINT CONVERSION')) & (combined_df['Description'].str.contains('ATTEMPT SUCCEEDS'))) | (combined_df['IsTwoPointConversionSuccessful'] == 1)
combined_df['IsTwoPointConversionScore'] = np.where(mask_4, 1, 0)

mask_5 = combined_df['Description'].str.contains('SAFETY')
combined_df['IsSafety'] = np.where(mask_5, 1, 0)

In [42]:
# Create new columns to assign score values on scoring plays
combined_df['IsHome'] = np.where(combined_df['Home_team'] == combined_df['OffenseTeam'], 1, 0)
combined_df['Home_score'] = 0
combined_df['Away_score'] = 0

mask_home = (combined_df['IsTouchdown'] == 1) & (combined_df['IsHome'] == 1)
combined_df.loc[mask_home, 'Home_score'] = 6
mask_away = (combined_df['IsTouchdown'] == 1) & (combined_df['IsHome'] == 0)
combined_df.loc[mask_away, 'Away_score'] = 6

mask_home = (combined_df['IsFieldGoal'] == 1) & (combined_df['IsHome'] == 1)
combined_df.loc[mask_home, 'Home_score'] = 3
mask_away = (combined_df['IsFieldGoal'] == 1) & (combined_df['IsHome'] == 0)
combined_df.loc[mask_away, 'Away_score'] = 3

mask_home = (combined_df['IsExtraPoint'] == 1) & (combined_df['IsHome'] == 1)
combined_df.loc[mask_home, 'Home_score'] = 1
mask_away = (combined_df['IsExtraPoint'] == 1) & (combined_df['IsHome'] == 0)
combined_df.loc[mask_away, 'Away_score'] = 1

mask_home = (combined_df['IsTwoPointConversionScore'] == 1) & (combined_df['IsHome'] == 1)
combined_df.loc[mask_home, 'Home_score'] = 2
mask_away = (combined_df['IsTwoPointConversionScore'] == 1) & (combined_df['IsHome'] == 0)
combined_df.loc[mask_away, 'Away_score'] = 2

mask_home = (combined_df['IsSafety'] == 1) & (combined_df['IsHome'] == 1)
combined_df.loc[mask_home, 'Home_score'] = 2
mask_away = (combined_df['IsSafety'] == 1) & (combined_df['IsHome'] == 0)
combined_df.loc[mask_away, 'Away_score'] = 2

In [43]:
# Merge df with coach data. Had to keep them separate to make sure offensive/defensive 
# coaches/coordinators are associated with the appropriate plays
off_df = pd.read_csv('../data/offensive_coach_data.csv')
def_df = pd.read_csv('../data/defensive_coach_data.csv')

off_df['SeasonYear'] = off_df['SeasonYear'].astype(float)
def_df['SeasonYear'] = off_df['SeasonYear'].astype(float)
new_combined_df = pd.merge(combined_df, off_df, how='left', left_on=['SeasonYear', 'OffenseTeam'], right_on=['SeasonYear', 'OffenseTeam'])
new_combined_df = pd.merge(new_combined_df, def_df, how='left', left_on=['SeasonYear', 'DefenseTeam'], right_on=['SeasonYear', 'DefenseTeam'])

In [44]:
# With the previously created Home_score/Away_score features, 
# calculate running totals for all plays. Sort by quarter, minute, second.
# Additionally, add feature Last_play and Two_plays_ago
new_combined_df['Home_total_score'] = None
new_combined_df['Away_total_score'] = None
new_combined_df['Last_play'] = None

unique_games = new_combined_df['GameId'].unique()
for game in unique_games:
    mask = new_combined_df['GameId'] == game
    game_df = new_combined_df[mask]
    game_df = game_df.sort_values([u'Quarter', u'Minute', u'Second'])
    
    team1, team2 = game_df['OffenseTeam'].unique()
    team1_plays = (game_df['OffenseTeam'] == team1) & (game_df['Down'] != 0)
    team2_plays = (game_df['OffenseTeam'] == team2) & (game_df['Down'] != 0)
    
    new_combined_df.loc[mask, 'Home_total_score'] = game_df['Home_score'].cumsum()
    new_combined_df.loc[mask, 'Away_total_score'] = game_df['Away_score'].cumsum()
    new_combined_df.loc[mask & team1_plays, 'Last_play'] = game_df[team1_plays]['PlayType'].shift(1)
    new_combined_df.loc[mask & team2_plays, 'Last_play'] = game_df[team2_plays]['PlayType'].shift(1)
    new_combined_df.loc[mask & team1_plays, 'Two_plays_ago'] = game_df[team1_plays]['PlayType'].shift(2)
    new_combined_df.loc[mask & team2_plays, 'Two_plays_ago'] = game_df[team2_plays]['PlayType'].shift(2)

In [45]:
# Create Score_differential to capture current leads and deficits
new_combined_df['Score_differential'] = 0

mask = new_combined_df['IsHome'] == 1
new_combined_df.loc[mask, 'Score_differential'] = new_combined_df['Home_total_score'] - new_combined_df['Away_total_score']

mask = new_combined_df['IsHome'] == 0
new_combined_df.loc[mask, 'Score_differential'] = new_combined_df['Away_total_score'] - new_combined_df['Home_total_score']

In [46]:
# Create features Last_play_IsPass, Last_two_plays_IsPass (0,1,2)
last_pass_mask = new_combined_df['Last_play'] == 'PASS'
new_combined_df['Last_play_IsPass'] = np.where(last_pass_mask, 1, 0)

new_combined_df['Last_two_plays_IsPass'] = 0
two_pass_mask = (last_pass_mask) & (new_combined_df['Two_plays_ago'] == 'PASS')
new_combined_df.loc[two_pass_mask, 'Last_two_plays_IsPass'] = 2

one_pass_mask = (last_pass_mask) & (new_combined_df['Two_plays_ago'] != 'PASS')
new_combined_df.loc[one_pass_mask, 'Last_two_plays_IsPass'] = 1

In [47]:
# Make new features, left_in_half , left_in_game (both in seconds)
new_combined_df['Left_in_game'] = 3600 - (((new_combined_df['Quarter'] - 1) * 900) + (new_combined_df['Minute'] * 60) + new_combined_df['Second'])

first_half_mask = (new_combined_df['Quarter'] == 1) | (new_combined_df['Quarter'] == 2)
new_combined_df.loc[first_half_mask, 'Left_in_half'] = 1800 - (((new_combined_df['Quarter'] - 1) * 900) + (new_combined_df['Minute'] * 60) + new_combined_df['Second'])

second_half_mask = (new_combined_df['Quarter'] == 3) | (new_combined_df['Quarter'] == 4) | (new_combined_df['Quarter'] == 5)
new_combined_df.loc[second_half_mask, 'Left_in_half'] = new_combined_df['Left_in_game']

In [48]:
# Extract month from gamedate
new_combined_df['Month'] = new_combined_df['GameDate'].apply(lambda x: x.month)

In [49]:
# Turn features into string format to dummify
new_combined_df['SeasonYear'] = new_combined_df['SeasonYear'].astype(int).astype(str)
new_combined_df['Quarter'] = new_combined_df['Quarter'].astype(str)
new_combined_df['Down'] = new_combined_df['Down'].astype(str)
new_combined_df['Month'] = new_combined_df['Month'].astype(str)

In [58]:
# new_combined_df.to_csv('../data/master.csv', index=False)